## Начинаем с импорта библиотек:

In [2]:
import pandas as pd
import psycopg2
import warnings

warnings.filterwarnings('ignore', category=UserWarning, 
                        message='pandas only supports SQLAlchemy connectable')

## параметры подключения будут следующими:

In [3]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

## Создаём соединение с заданными параметрами:

In [4]:
# Создаём соединение с заданными параметрами:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

Мы подключаемся к базе данных 

### Таблицы:




### vacancies (вакансии)

Таблица хранит в себе данные по вакансиям и содержит следующие столбцы:


| Поле             | Тип  | Флаг | Комментарий                       |
|------------------|------|------|-----------------------------------|
| 123 id           | int4 | [V]  | ID вакансии                      |
| RBC name         | text | [V]  | Название вакансии                |
| RBC key_skills   | text | [ ]  | Ключевые навыки                  |
| RBC schedule     | text | [V]  | Тип рабочего графика             |
| RBC experience   | text | [V]  | Требования к опыту               |
| RBC employment   | text | [V]  | Тип трудоустройства              |
| 123 salary_from  | int4 | [ ]  | Нижняя граница зарплатной вилки  |
| 123 salary_to    | int4 | [ ]  | Верная граница зарплатной вилки  |
| 123 area_id      | int4 | [V]  | ID региона вакансии              |
| 123 employer_id  | int4 | [V]  | ID работодателя                  |



### areas (регионы)
Таблица-справочник, которая хранит код региона и его название.

| Название     | Тип  данных | Not Null |  Комментарий              |
|----------|------|------|-----------------------|
| id   | int4 | [v]  | ID  региона  |
| name     | text | [v]  | Название региона      |



#### employers (работодатели)

Таблица-справочник со списком работодателей.

| Название     | Тип  данных | Not Null |  Комментарий              |
|----------|------|------|-----------------------|
| id   | int4 | [v]  | ID  города  |
| name     | text | [v]  | Название работодателя      |
| area     | int4 | [v]  | Регион регистрации  (ссылка на areas.id)     |



### industries (сферы деятельности)
Таблица-справочник вариантов сфер деятельности работодателей.

| Название     | Тип  данных | Not Null |  Комментарий              |
|----------|------|------|-----------------------|
| id   | varchar(10) | [v]  | ID  сферы деятельности  |
| name     | text | [v]  |  Название сферы деятельности     |





### Связи:

employers.area -> areas.id

vacancies.area_id -> areas.id

vacancies.employer_id -> employers.id

employers_industries.employer_id -> employers.id

employers_industries.industry_id -> industries.id

## Предварительный анализ данных

Для начала познакомимся поближе с данными в таблицах. Напишите запросы, которые позволят ответить на вопросы ниже. В поле для ввода укажите число, которое является ответом на вопрос, а в ноутбук-решение запишите сами запросы.

### Задание 1

Сколько вакансий есть в базе?

In [22]:
query = '''

SELECT 
    (SELECT COUNT(*) FROM vacancies) as vacancies_count;
    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'Вакансий в базе: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Вакансий в базе: 49197




### Задание 2

Сколько работодателей?

In [21]:
query = '''

SELECT 
    (SELECT COUNT(*) FROM employers) as employers_count;
    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'Работодателей в базе: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Работодателей в базе: 23501




### Задание 3

Сколько регионов?

In [20]:
query = '''

SELECT 
    (SELECT COUNT(*) FROM areas) as areas_count;
    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'Регионов в базе: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Регионов в базе: 1362




### Задание 4

Сколько сфер деятельности?

In [19]:
query = '''


SELECT 
    (SELECT COUNT(*) FROM industries) as industries_count;

    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'Сфер деятельности в базе: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Сфер деятельности в базе: 294




## Детальный анализ вакансий

### Задание 1

Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

Выберите пятёрку лидеров по количеству вакансий:

In [52]:
query = '''
SELECT 
    a.name AS area,
    COUNT(v.id) AS cnt
FROM areas a
JOIN vacancies v ON a.id = v.area_id
GROUP BY a.id, a.name
ORDER BY cnt DESC
LIMIT 5;
'''

result = pd.read_sql_query(query, connection)
cities_string = ', '.join(result['area'].tolist())

print("\n" + "="*60 + "\n")
print(cities_string)
print("\n" + "="*60 + "\n")



Москва, Санкт-Петербург, Минск, Новосибирск, Алматы




### Задание 2

Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [41]:
query = '''
SELECT COUNT(*) 
FROM vacancies 
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;
'''

result = pd.read_sql_query(query, connection)


print("\n" + "="*60 + "\n")
print(f'Количество вакансий заполнено хотя бы одно из двух полей с зарплатой: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Количество вакансий заполнено хотя бы одно из двух полей с зарплатой: 24073




### Задание 3

Найдите средние значения для нижней и верхней границы зарплатной вилки.

Округлите значения до целого числа.

In [37]:
query = '''
SELECT 
     ROUND(AVG(salary_from)) AS avg_salary_from,
     ROUND(AVG(salary_to)) AS avg_salary_to
 FROM vacancies
 WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'Нижняя граница: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")
print(f'Верхняя граница: {result.iat[0, 1]}')
print("\n" + "="*60 + "\n")



Нижняя граница: 71065.0


Верхняя граница: 110537.0




### Задание 4

Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

Какая пара находится на втором месте по популярности?

In [51]:
query = '''
SELECT 
    schedule,
    employment,
    COUNT(*) AS vacancy_count
FROM vacancies
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
LIMIT 1 OFFSET 1;
'''

result = pd.read_sql_query(query, connection)

cities_string = combined_string = ', '.join(result.apply(lambda row: f"{row['schedule']}, {row['employment']}", axis=1))

print("\n" + "="*60 + "\n")
print(cities_string)
print("\n" + "="*60 + "\n")



Удаленная работа, Полная занятость




### Задание 5

Напишите запрос, выводящий значения поля «Требуемый опыт работы» (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.


In [50]:
query = '''SELECT 
    experience,
    COUNT(*) AS vacancy_count
FROM vacancies
GROUP BY experience
ORDER BY vacancy_count ASC;
'''

result = pd.read_sql_query(query, connection)

experience_string = ', '.join(result['experience'].tolist())
print("\n" + "="*60 + "\n")
print(experience_string)
print("\n" + "="*60 + "\n")



Более 6 лет, Нет опыта, От 3 до 6 лет, От 1 года до 3 лет




## Анализ работодателей

### Задание 1

Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.


In [56]:
query = '''WITH employer_ranks AS (
    SELECT 
        e.id,
        e.name,
        COUNT(v.id) AS vacancy_count,
        DENSE_RANK() OVER (ORDER BY COUNT(v.id) DESC) AS rank_position
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
)
SELECT 
    name,
    vacancy_count,
    rank_position
FROM employer_ranks
WHERE rank_position = 1 OR rank_position = 5
ORDER BY rank_position;

'''

result = pd.read_sql_query(query, connection)



name_string = ', '.join(result['name'].tolist())

print("\n" + "="*60 + "\n")
print(name_string)
print("\n" + "="*60 + "\n")



Яндекс, Газпром нефть




### Задание 2

Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [59]:
query = '''
SELECT a.name
FROM areas a
LEFT JOIN employers e ON a.id = e.area
LEFT JOIN vacancies v ON a.id = v.area_id
GROUP BY a.id, a.name
HAVING COUNT(v.id) = 0
ORDER BY COUNT(DISTINCT e.id) DESC
LIMIT 1;

'''

result = pd.read_sql_query(query, connection)


print("\n" + "="*60 + "\n")
print(f'Регион: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")



Регион: Россия




### Задание 3

Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

In [77]:
query = '''
WITH employer_region_counts AS (
    SELECT 
        e.id,
        e.name,
        COUNT(DISTINCT v.area_id) AS region_count
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
)
SELECT MAX(region_count) AS max_regions_per_employer
FROM employer_region_counts;

'''

result = pd.read_sql_query(query, connection)


print("\n" + "="*60 + "\n")
print(f'Количество регионов, в которых он публикует свои вакансии: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")
print("\nПолный результат:\n")
print(result)



Количество регионов, в которых он публикует свои вакансии: 181



Полный результат:

   max_regions_per_employer
0                       181


### Задание 4

Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

Введите количество, которое вернул запрос, в поле ниже.

In [76]:
query = '''
SELECT COUNT(*) AS employers_without_industry
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.employer_id IS NULL;

'''

result = pd.read_sql_query(query, connection)


print("\n" + "="*60 + "\n")
print(f'Количества работодателей, у которых не указана сфера деятельности: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")
print("\nПолный результат:\n")
print(result)



Количества работодателей, у которых не указана сфера деятельности: 8419



Полный результат:

   employers_without_industry
0                        8419


### Задание 5

Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

Введите в поле ниже название этой компании так же, как оно указано в результате запроса.

In [75]:
query = '''
WITH employers_with_4_industries AS (
    SELECT 
        e.id,
        e.name,
        COUNT(ei.industry_id) AS industry_count
    FROM employers e
    INNER JOIN employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.id, e.name
    HAVING COUNT(ei.industry_id) = 4
)
SELECT name
FROM employers_with_4_industries
ORDER BY name, id
LIMIT 1 OFFSET 2;

'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")
print("\nПолный результат:\n")
print(result)



название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности: 2ГИС



Полный результат:

   name
0  2ГИС


### Задание 6


С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [74]:
query = '''
SELECT COUNT(DISTINCT employer_id) AS employers_count
FROM employers_industries
WHERE industry_id = (
    SELECT id 
    FROM industries 
    WHERE name = 'Разработка программного обеспечения'
);

'''

result = pd.read_sql_query(query, connection)


print("\n" + "="*60 + "\n")
print(f'количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")
print("\nПолный результат:\n")
print(result)



количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения»: 3553



Полный результат:
   employers_count
0             3553


### Задание 7

Для компании «Яндекс» выведите список городов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании, собранных в этой таблице. Должна получиться выборка такого вида (приведён пример результата для компании SberTech):

In [79]:
query = '''
WITH city_counts AS (
    SELECT 
        a.name AS city,
        COUNT(*) AS cnt
    FROM vacancies v
    JOIN employers e ON v.employer_id = e.id
    JOIN areas a ON v.area_id = a.id
    WHERE e.name = 'Яндекс'
      AND a.name IN (
          'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 
          'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 
          'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 
          'Воронеж', 'Пермь', 'Волгоград'
      )
    GROUP BY a.name
),
combined AS (
    SELECT city, cnt FROM city_counts
    UNION ALL
    SELECT 'Total', COALESCE(SUM(cnt), 0) FROM city_counts
)
SELECT city AS "ABC name", cnt AS "123 cnt"
FROM combined
ORDER BY 
    CASE WHEN city = 'Total' THEN 1 ELSE 0 END,
    cnt,
    city;
'''

result = pd.read_sql_query(query, connection)


# Количество строк (включая Total)
num_rows = len(result)

print("\n" + "="*60 + "\n")
print(f"Количество строк: {num_rows}")
print("\n" + "="*60 + "\n")

# Значение Total
print("\n" + "="*60 + "\n")
total_value = result.loc[result['ABC name'] == 'Total', '123 cnt'].values[0]
print(f"Значение Total: {total_value}")
print("\n" + "="*60 + "\n")

print("\nПолный результат:")
print(result)



Количество строк: 17




Значение Total: 485.0



Полный результат:
           ABC name  123 cnt
0              Омск     21.0
1         Челябинск     22.0
2        Красноярск     23.0
3         Волгоград     24.0
4            Казань     25.0
5             Пермь     25.0
6    Ростов-на-Дону     25.0
7            Самара     26.0
8               Уфа     26.0
9         Краснодар     30.0
10          Воронеж     32.0
11      Новосибирск     35.0
12  Нижний Новгород     36.0
13     Екатеринбург     39.0
14  Санкт-Петербург     42.0
15           Москва     54.0
16            Total    485.0


## Предметный анализ

Переходим к самому интересному — нам предстоит исследовать требования работодателей к дата-сайентистам.


### Задание 1

Сколько вакансий имеет отношение к данным?

In [81]:
query = '''SELECT COUNT(*) AS count
FROM vacancies
WHERE name ILIKE '%data%' OR name ILIKE '%данн%';
   '''

result = pd.read_sql_query(query, connection)
print("\n" + "="*60 + "\n")
print(f'вакансий имеет отношение к данным: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result)



вакансий имеет отношение к данным: 1771



Полный результат:
   count
0   1771


### Задание 2

Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

* ‘data scientist’;
* ‘data science’;
* ‘исследователь данных’;
* ‘ML’ (здесь не нужно брать вакансии по HTML);
* ‘machine learning’;
* ‘машинн%обучен%’.

В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Считаем вакансиями для специалистов уровня Junior следующие:

* в названии есть слово “junior” или
* требуемый опыт — «Нет опыта» или
* тип трудоустройства — «Стажировка».
* 
В качестве ответа запишите количество вакансий, которое вернул запрос.

In [82]:
query = '''SELECT COUNT(*)
FROM vacancies
WHERE 
  -- Условие для дата-сайентистов
   (name ILIKE '%data scientist%' OR
   name ILIKE '%data science%' OR
   name ILIKE '%исследователь данных%' OR
   (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
   name ILIKE '%machine learning%' OR
   name ILIKE '%машинн%обучен%')
   -- Условие для Junior
   AND
   (name ILIKE '%junior%' OR
   experience = 'Нет опыта' OR
   employment = 'Стажировка')
 '''

result = pd.read_sql_query(query, connection)
print("\n" + "="*60 + "\n")
print(f'подходящих вакансий для начинающего дата-сайентиста: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result)     



подходящих вакансий для начинающего дата-сайентиста: 51



Полный результат:
   count
0     51


### Задание 3

Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

В качестве ответа запишите количество, которое вернул запрос.

In [83]:
query = '''

SELECT COUNT(*) as count
FROM vacancies
WHERE 
    -- Условия для вакансий Data Science
    (
        LOWER(name) LIKE '%data scientist%' OR
        LOWER(name) LIKE '%data science%' OR
        LOWER(name) LIKE '%исследователь данных%' OR
        (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
        LOWER(name) LIKE '%machine learning%' OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    -- Условие по ключевым навыкам: наличие SQL или Postgres
    AND (
        LOWER(key_skills) LIKE '%sql%' OR
        LOWER(key_skills) LIKE '%postgres%'
    );
    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'подходящих вакансий для начинающего дата-сайентиста: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result) 



подходящих вакансий для начинающего дата-сайентиста: 229



Полный результат:
   count
0    229


### Задание 4

С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

В качестве ответа запишите количество, которое вернул запрос.

In [85]:
query = '''


    SELECT COUNT(*) as count
    FROM vacancies
    WHERE 
        (
            LOWER(name) LIKE '%data scientist%' OR
            LOWER(name) LIKE '%data science%' OR
            LOWER(name) LIKE '%исследователь данных%' OR
            (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
            LOWER(name) LIKE '%machine learning%' OR
            LOWER(name) LIKE '%машинн%обучен%'
        )
        AND LOWER(key_skills) LIKE '%python%';

'''



result = pd.read_sql_query(query, connection)
print("\n" + "="*60 + "\n")
print(f'насколько популярен Python в требованиях работодателей к DS: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result) 



насколько популярен Python в требованиях работодателей к DS: 357



Полный результат:
   count
0    357


### Задание 5

Сколько ключевых навыков в среднем указывают в вакансиях для DS?

Ответ округлите до двух знаков после точки-разделителя.

In [87]:
query = '''

SELECT 
    ROUND(
        AVG(
            array_length(
                array_remove(string_to_array(key_skills, E'\\t'), ''),
                1
            )
        )::numeric, 
    2) as avg_skills_count
FROM vacancies
WHERE 
    -- Условия для вакансий DS
    (name ILIKE '%data scientist%' OR
     name ILIKE '%data science%' OR
     name ILIKE '%исследователь данных%' OR
     (name ILIKE '%ml%' AND name NOT ILIKE '%html%') OR
     name ILIKE '%machine learning%' OR
     name ILIKE '%машинн%обучен%')
    -- Исключаем вакансии без ключевых навыков
    AND key_skills IS NOT NULL 
    AND key_skills != '';

    
'''

result = pd.read_sql_query(query, connection)
print("\n" + "="*60 + "\n")
print(f'ключевых навыков в среднем указывают в вакансиях для DS: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result)



ключевых навыков в среднем указывают в вакансиях для DS: 6.55



Полный результат:
   avg_skills_count
0              6.55


### Задание 6

Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).

При решении задачи примите во внимание следующее:

1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null).
Чтобы избежать этой ситуации, мы воспользуемся функцией coalesce, которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос select 1 + coalesce(null, 0).
Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [88]:
query = '''

SELECT 
    ROUND(AVG(
        CASE 
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2.0
            WHEN salary_from IS NOT NULL THEN salary_from::float
            ELSE salary_to::float
        END
    )) AS avg_salary
FROM vacancies
WHERE 
    -- условия для DS
    (name ILIKE '%data scientist%' OR
     name ILIKE '%data science%' OR
     name ILIKE '%исследователь данных%' OR
     (name ILIKE '%ml%' AND name NOT ILIKE '%html%') OR
     name ILIKE '%machine learning%' OR
     name ILIKE '%машинн%обучен%')
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    AND experience ILIKE '%от 3 до 6 лет%';
    
'''

result = pd.read_sql_query(query, connection)

print("\n" + "="*60 + "\n")
print(f'зарплата в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет: {result.iat[0, 0]}')
print("\n" + "="*60 + "\n")


print("\nПолный результат:")
print(result)



зарплата в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет: 256454.0



Полный результат:
   avg_salary
0    256454.0


In [89]:
# не забываем закрыть соединение после окончания работы
connection.close()